In [1]:
import json
import pandas as pd
from pymongo import MongoClient
from functions.mongo import connectCollection

Insert your AtlasMongoDB Admin password: ········


In [2]:
db, coll = connectCollection('api-project','chats')
with open('./input/chats.json', encoding="utf8") as f:
    chats_json = json.load(f)
coll.insert_many(chats_json)

In [3]:
df = pd.read_json('./input/chats.json', orient='records')

In [4]:
chatsDF = df[['idChat', 'idUser']]
chatsDF.drop_duplicates(inplace=True)
chatsDF.reset_index(drop=True, inplace=True)
uniqueChats = list(set(chatsDF['idChat']))
users_total = []
for e in uniqueChats:
    users = []
    for i in range(len(chatsDF)):
        if chatsDF.at[i,'idChat'] == e:
            users.append(chatsDF.at[i,'idUser'])
    users_total.append(str(users))
chatsDF = pd.DataFrame(uniqueChats, columns=['idChat'])
chatsDF['users'] = users_total
coll4 = db['chatsCR']
coll4.insert_many(chatsDF.to_dict('records'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
dfNew = df[['userName', 'idUser']]

In [6]:
dfNew = dfNew[['userName', 'idUser']].mask(dfNew[['userName', 'idUser']].duplicated())

In [7]:
dfNew = dfNew.dropna()

In [8]:
dfNew.to_json('./output/users.json', orient='records')

In [9]:
coll2 = db['users']
coll2.insert_many(dfNew.to_dict('records'))

In [15]:
dfchats.to_json("./output/justchats.json", orient='records')

In [16]:
db, coll = connectCollection('api-project','users')
data = list(coll.find({}))
user_id_list = []
for i in range(len(df)):
    for e in data:
        if df.at[i,'idUser'] == e['idUser']:
            user_id_list.append(e['_id'])
df['user_id'] = user_id_list
coll4 = db['messages']
coll4.insert_many(df.to_dict('records'))